<a href="https://colab.research.google.com/github/Zelechos/Generador-Css/blob/main/PruebaRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En este post vamos a entrenar una `red neuronal recurrente` para generar texto, carácter a carácter, inspirado en [CharRNN](https://github.com/karpathy/char-rnn). Nuestra red neuronal recibirá como entrada una secuencia de letras y deberá dar como salida la siguiente letra (la cual añadiremos a las entradas para volver a generar un nuevo carácter). 

## Los datos

Lo primero que necesitamos para lograr nuestro objetivo es un conjunto de datos. En este caso, al querer generar texto, nos servirá con un archivo con mucho texto que queramos imitar. Para ello descargaremos *Don Quijote de la Mancha*, la obra principal del escritor Miguel de Cervantes y una de las más relevantes en la literatura castellana. 

In [ ]:
!git clone https://github.com/Zelechos/Generador-Css.git

Cloning into 'Generador-Css'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 61 (delta 23), reused 13 (delta 3), pack-reused 0
Unpacking objects: 100% (61/61), done.


In [ ]:
f = open("el_quijote.txt", "r", encoding='utf-8')
text = f.read()
text[:300], len(text)

('DON QUIJOTE DE LA MANCHA\nMiguel de Cervantes Saavedra\n\nPRIMERA PARTE\nCAPÍTULO 1: Que trata de la condición y ejercicio del famoso hidalgo D. Quijote de la Mancha\nEn un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho tiempo que vivía un hidalgo de los de lanza en astillero, ada',
 1038397)

In [ ]:
import json

with open('/content/Generador-Css/dataset/TEST.json') as file:
    data = json.load(file)

#accedemos a la los datos
data_styles = data["styles"]
print(data_styles[0])


#accediendo a los estilos
styles = data_styles[0]["style"]
print("\nstyles access => ",styles)


#accediendo a los selectores
selector = data_styles[0]["selector"]
print("\nselector access => ",selector)

text = ""
for styles in data_styles:
  # print(styles['style'])
  text += styles['style']

print(text)

{'style': 'body{overflow:hidden;background:rgb(25,35,125)}', 'selector': 'body'}

styles access =>  body{overflow:hidden;background:rgb(25,35,125)}

selector access =>  body
body{overflow:hidden;background:rgb(25,35,125)}div.drop-container{position:absolute;top:0;right:0;bottom:0;left:0;margin:auto;height:200px;width:200px}div.drop{position:absolute;top:-25%;width:100%;height:100%;border-radius:100% 5% 100% 100%;-webkit-transform:rotate(-45deg);transform:rotate(-45deg);margin:0;background:deepskyblue;-webkit-animation:drip 4s forwards;animation:drip 4s forwards}h1{color:#fff;position:absolute;font-size:2.5em;height:1em;top:0;left:0;right:0;bottom:0;z-index:2;margin:auto;text-align:center;opacity:0;-webkit-animation:appear 2s 2.5s forwards;animation:appear 2s 2.5s forwards}@font-face {font-family: logofont;src: url(https://s3-us-west-2.amazonaws.com/s.cdpn.io/531144/couture-bld.otf);}body {margin: 0px;background: #EF5350;}.logo {transform: scale(0.7);margin-top: 45px;}.switch-left{posit

Tenemos alrededor de 1 millón de carácteres en nuestro dataset, suficientes para generar texto de manera convincente como si fuésemos el manco de Lepanto.

## Tokenización

Para poder darle este texto a nuestra red neuronal necesitamos transformarlo en números con los que podemos llevar a cabo las operaciones que tienen lugar en la red. Este proceso se conoce como `tokenización`. Existen muchas formas de llevar a cabo este proceso, en este caso simplemente sustituiremos cada carácter en nuestro texto por su posición en el siguiente vector de carácteres.

In [ ]:
import string

all_characters = string.printable + "ñÑáÁéÉíÍóÓúÚ¿¡"
all_characters

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0cñÑáÁéÉíÍóÓúÚ¿¡'

In [ ]:
import string

class Tokenizer(): 
    
  def __init__(self):
    self.all_characters = all_characters
    self.n_characters = len(self.all_characters)
    
  def text_to_seq(self, string):
    seq = []
    for c in range(len(string)):
        try:
            seq.append(self.all_characters.index(string[c]))
        except:
            continue
    return seq

  def seq_to_text(self, seq):
    text = ''
    for c in range(len(seq)):
        text += self.all_characters[seq[c]]
    return text

tokenizer = Tokenizer()
tokenizer.n_characters

114

El tokenizer puede convertir una secuencia de texto en números, y al revés.

In [ ]:
tokenizer.text_to_seq('señor, ¿qué tal?')

[28, 14, 100, 24, 27, 73, 94, 112, 26, 30, 104, 94, 29, 10, 21, 82]

In [ ]:
tokenizer.seq_to_text([28, 14, 100, 24, 27, 73, 94, 112, 26, 30, 104, 94, 29, 10, 21, 82])

'señor, ¿qué tal?'

Ahora podemos tokenizar todo el texto.

In [ ]:
text_encoded = tokenizer.text_to_seq(text)

> 💡 Pese a que podemos implementar nuestra lógica de tokenización para trabajar a nivel de carácteres, cuando trabajamos con palabras completas el proceso puede complicarse. Es por esto que existen muchas herramientas que ya implementan este tipo de procesado (y muchos otros) que podemos utilizar. Un ejemplo, especialmente integrado con `Pytorch`, es la librería [torchtext](https://pytorch.org/text/).

## El *Dataset*

En primer lugar, vamos a separar nuestro texto en un conjunto de entrenamiento y otro de test. Cómo ya hemos hablado en posts anteriores, usaremos los datos de entrenamiento para entrenar nuestra red neuronal y los datos de test para calcular las métricas finales.

In [ ]:
train_size = len(text_encoded) * 80 // 100 
train = text_encoded[:train_size]
test = text_encoded[train_size:]

len(train), len(test)

(6768, 1693)

Para entrenar nuestra red, vamos a necesitar secuencias de texto de una longitud determinada. Podemos generar estas ventanas con la siguiente función

In [ ]:
import random

def windows(text, window_size = 100):
    start_index = 0
    end_index = len(text) - window_size
    text_windows = []
    while start_index < end_index:
      text_windows.append(text[start_index:start_index+window_size+1])
      start_index += 1
    return text_windows

text_encoded_windows = windows(text_encoded)

Como puedes ver, hemos generado un número determinado de frases con la longitud especificada las cuales empiezan cada vez un carácter más a la derecha.

In [ ]:
print(tokenizer.seq_to_text((text_encoded_windows[0])))
print()
print(tokenizer.seq_to_text((text_encoded_windows[1])))
print()
print(tokenizer.seq_to_text((text_encoded_windows[2])))

body{overflow:hidden;background:rgb(25,35,125)}div.drop-container{position:absolute;top:0;right:0;bot

ody{overflow:hidden;background:rgb(25,35,125)}div.drop-container{position:absolute;top:0;right:0;bott

dy{overflow:hidden;background:rgb(25,35,125)}div.drop-container{position:absolute;top:0;right:0;botto


Nuestro *dataset* de `Pytorch` se encargará de darnos cada una de estas frases, utilizando todos los carácteres excepto el último como entradas para la red y el último carácter como la etiqueta que usaremos durante el entrenamiento (la red deberá predecir la siguiente letra).

In [ ]:
import torch

class CharRNNDataset(torch.utils.data.Dataset):
  def __init__(self, text_encoded_windows, train=True):
    self.text = text_encoded_windows
    self.train = train

  def __len__(self):
    return len(self.text)

  def __getitem__(self, ix):
    if self.train:
      return torch.tensor(self.text[ix][:-1]), torch.tensor(self.text[ix][-1])
    return torch.tensor(self.text[ix])

In [ ]:
train_text_encoded_windows = windows(train)
test_text_encoded_windows = windows(test)

dataset = {
    'train': CharRNNDataset(train_text_encoded_windows),
    'val': CharRNNDataset(test_text_encoded_windows)
}

dataloader = {
    'train': torch.utils.data.DataLoader(dataset['train'], batch_size=512, shuffle=True, pin_memory=True),
    'val': torch.utils.data.DataLoader(dataset['val'], batch_size=2048, shuffle=False, pin_memory=True),
}

len(dataset['train']), len(dataset['val'])

(6668, 1593)

In [ ]:
input, output = dataset['train'][0]
tokenizer.seq_to_text(input)

'body{overflow:hidden;background:rgb(25,35,125)}div.drop-container{position:absolute;top:0;right:0;bo'

In [ ]:
tokenizer.seq_to_text([output])

't'

## Embeddings

Si bien hemos conseguido convertir nuestro texto a números, una red neuronal seguirá sin ser capaz de trabajar con nuestros datos ya que, como hemos visto en posts anteriores, éstos tienen que estar normalizados. Además, en función del `tokenizador` que utilicemos es posible que el  mismo carácter tenga asociados diferentes valores. Es por esto que necesitamos codificar nuestro texto de alguna manera. 

Una opción puede ser el `one-hot encoding`, al fin y al cabo podemos considerar cada letra como una categoría y que nuestra red nos de a la salida una distribución de probabilidad sobre todos los posibles carácteres. A continuación tienes un ejemplo de este tipo de codificación (utilizando palabras en vez de letras).

![](https://i0.wp.com/shanelynnwebsite-mid9n9g1q9y8tt.netdna-ssl.com/wp-content/uploads/2018/01/one-hot-word-embedding-vectors.png?ssl=1)

A nuestra red le daremos a la entrada un vector que representará cada elemento en el vocabulario. Este vector tendrá una longitud igual al número de elementos diferentes en el vocabulario, y estará lleno de ceros excepto por una posición (la posición que ocupe el elemento en concreto dentro del vocabulario, la lista de elementos únicos). En nuestro caso podríamos optar por esta alternativa, ya que apenas tenemos un centenar de carácteres diferentes. Sin embargo, cuando trabajemos con palabras, nuestros vocabularios serán enormes (¿cuántas palabras hay en el diccionario?). Esto implica que trabajar con una codificación `one-hot` será muy costoso (vectores muy grandes) e ineficiente (prácticamente llenos de ceros). Es por esto que utilizamos una mejor codificación: los `embeddings`

![](https://i.stack.imgur.com/5gAnY.png)

Un embedding es una matriz con un número de filas igual al tamaño del vocabulario y un número de columnas que nosotros decidiremos. Cada fila en la matriz representará la codificación de una palabara (o carácter en nuestro ejemplo). A diferencia de la codificación `one-hot`, estos vectores son densos (pueden tener valores diferentes de cero en cualquier posición). Además, estos valores son aprendidos por la red neuronal, de manera que podrá representar los datos de la mejor forma posible para llevar a cabo la tarea. En la figura anterior tienes un ejemplo de un embedding entrenado, ¿observas algún patrón?. Efectivamente, palabras similares tienen representaciones similares. Además, cada columna del embedding tiene un significado que permite establecer relaciones entre las diferentes representaciones.

> ⚡ ¿Qué resultado obtienes al restar el vector `boy` al vector `man` y sumarle el vector `girl`?

En `Pytorch` tenemos esta capa implementada en la clase `torch.nn.Embedding`, y más adelante veremos como podemos utilizar `transfer learning` con embeddings pre-entrenados (lo cual nos dará una mejor representación de nuestro vocabulario desde el principio sin tener que entrenar esta capa).

In [ ]:
class CharRNN(torch.nn.Module):
  def __init__(self, input_size, embedding_size=128, hidden_size=256, num_layers=2, dropout=0.2):
    super().__init__()
    self.encoder = torch.nn.Embedding(input_size, embedding_size)
    self.rnn = torch.nn.LSTM(input_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers, dropout=dropout, batch_first=True)
    self.fc = torch.nn.Linear(hidden_size, input_size)

  def forward(self, x):
    x = self.encoder(x)
    x, h = self.rnn(x)         
    y = self.fc(x[:,-1,:])
    return y

Nuestro modelo recibirá *batches* de frases con el índice de cada palabra que nos proporciona el `tokenizador`. A la salida tendremos una distribución de probabilidad sobre todos los posibles carácteres para cada frase del *batch*. Aquellos con mayor probabilidad serán los que la red cree que son buenos candidatos para seguir la frase recibida a la entrada.

In [ ]:
model = CharRNN(input_size=tokenizer.n_characters)
outputs = model(torch.randint(0, tokenizer.n_characters, (64, 50)))
outputs.shape

torch.Size([64, 114])

## Entrenamiento

In [ ]:
from tqdm import tqdm
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

def fit(model, dataloader, epochs=10):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(1, epochs+1):
        model.train()
        train_loss = []
        bar = tqdm(dataloader['train'])
        for batch in bar:
            X, y = batch
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            y_hat = model(X)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
            bar.set_description(f"loss {np.mean(train_loss):.5f}")
        bar = tqdm(dataloader['val'])
        val_loss = []
        model.eval()
        with torch.no_grad():
            for batch in bar:
                X, y = batch
                X, y = X.to(device), y.to(device)
                y_hat = model(X)
                loss = criterion(y_hat, y)
                val_loss.append(loss.item())
                bar.set_description(f"val_loss {np.mean(val_loss):.5f}")
        print(f"Epoch {epoch}/{epochs} loss {np.mean(train_loss):.5f} val_loss {np.mean(val_loss):.5f}")

def predict(model, X):
    model.eval() 
    with torch.no_grad():
        X = torch.tensor(X).to(device)
        pred = model(X.unsqueeze(0))
        return pred

In [ ]:
model = CharRNN(input_size=tokenizer.n_characters)
fit(model, dataloader, epochs=20)

val_loss 3.56558: 100%|██████████| 1/1 [00:00<00:00,  4.93it/s]


Epoch 1/20 loss 4.12709 val_loss 3.56558


val_loss 3.49803: 100%|██████████| 1/1 [00:00<00:00,  5.12it/s]


Epoch 2/20 loss 3.62938 val_loss 3.49803


val_loss 3.43233: 100%|██████████| 1/1 [00:00<00:00,  5.13it/s]


...

Epoch 18/20 loss 1.71380 val_loss 1.92471


val_loss 1.88185: 100%|██████████| 1/1 [00:00<00:00,  5.55it/s]


Epoch 19/20 loss 1.62674 val_loss 1.88185


val_loss 1.82488: 100%|██████████| 1/1 [00:00<00:00,  5.06it/s]

Epoch 20/20 loss 1.56245 val_loss 1.82488


## Generando texto

Una vez hemos entrenado nuestro modelo, podemos darle una frase para que genere la siguiente letra.

In [ ]:
X_new = "body{"
X_new_encoded = tokenizer.text_to_seq(X_new)
y_pred = predict(model, X_new_encoded)
y_pred = torch.argmax(y_pred, axis=1)[0].item()
tokenizer.seq_to_text([y_pred])

'm'

Podemos generar más letras añadiendo las predicciones como parte de la entrada, generando texto letra a letra.

In [ ]:
for i in range(100):
  X_new_encoded = tokenizer.text_to_seq(X_new[-100:])
  y_pred = predict(model, X_new_encoded)
  y_pred = torch.argmax(y_pred, axis=1)[0].item()
  X_new += tokenizer.seq_to_text([y_pred])

X_new

'body{margin:1;transform:cale(-50% - 25px);border-radiun:1;margin:1;transform:cale(-50% - 25px);border-radiun:1;margin:1;transform:cale(-50% - 25px);border-radiun:1;margin:1;transform:cale(-50% - 25px);border-radiun:1;margin:1;transform:cale(-50% - 25px);border-radiun:1;margin:1;transform:cale(-50% - 25px'

Cómo puedes ver el text generado puede ser repetitivo si simplemente nos quedamos con la letra con mayor probabilidad. Para generar texto con mayor variedad, es común elegir de manera aleatoria una letra de entre las que tienen mayor probabilidad.

In [ ]:
temp=1
for i in range(1000):
  X_new_encoded = tokenizer.text_to_seq(X_new[-100:])
  y_pred = predict(model, X_new_encoded)
  y_pred = y_pred.view(-1).div(temp).exp()
  top_i = torch.multinomial(y_pred, 1)[0]
  predicted_char = tokenizer.all_characters[top_i]
  X_new += predicted_char

print(X_new)

body{margin:1;transform:cale(-50% - 25px);border-radiun:1;margin:1;transform:cale(-50% - 25px);border-radiun:1;margin:1;transform:cale(-50% - 25px);border-radiun:1;margin:1;transform:cale(-50% - 25px);border-radiun:1;margin:1;transform:cale(-50% - 25px);border-radiun:1;margin:1;transform:cale(-50% - 25px 210%}byht:iverhe 3s e(,ansity-rhebtcole:';zit-:calvc(0+9 lhc(1400  B'mÓ al(ac(lec(}zniteht:.5em50%;min-}ht-txan-seah-bolor:0ralyalh{m}ationo:.5{p}adkion:bow;let:2contemtracitent:1;banov-re-rader:0;posirifh-sitem:1;margin:}p:6c;derifttr{fort:0;pagngin:ttop:fixem;orex-tifteom:#ff;borkilatcfolce.nett;tixt-saty:2#5o#7.5mprian-coshtion-gimein:imentrayelhy1# sfffop:#35;bocko{tnanito{w-site:fopx;.js-sime:17;coms:1 0px;outton-oreatifon:.cex;wiith:lux;findexid-lation:cofornt-sraderean;sopady-le:fang,an-ciopne 0a3s o'tox:1;targhs2{1emtbith:4.v;lh-sizw:cote;grid:robe;height:.5px;tont--siation:heig;tiane:cele :atentolftefoc-oletn:6}purdips{pad-inggw:#-0}leftsan-peadis efon:camail{top:.5em;clorn:ce

## Resumen

En este post hemos aprendido cómo implementar y entrenar una `red neuronal recurrente` para generar texto como si fuese Miguel de Cervantes. Para ello hemos utilizado su libro *Don Quijote de la Mancha* como dataset. En primer lugar, transformamos el texto en números gracias al proceso de la `tokenización`. Después, codificamos cada carácter en el dataset utilizando una capa `embedding`, que permitirá a la red neuronal encontrar la mejor representación posible de los datos para llevar a cabo su tarea. Para generar texto, le pedimos a la red que nos de una distribución de probabilidad sobre todos los posible carácteres a partir de una frase que le damos a la entrada. Utilizaremos esta distribución para seleccionar un carácter que siga con la frase de manera convincente. Podemos repetir este proceso para generar secuencias más largas.